In [2]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers.pipelines.pt_utils import KeyDataset
#from datasets import load_dataset
import matplotlib.pyplot as plt
import torch 
import torch.nn.functional as F
import io
import seaborn as sns
import pandas as pd
import numpy as np
import glob
import os
import re
import datetime

In [73]:
# setting the path for joining multiple files
files = os.path.join("../Emilia/Raw(uncleaned) csv files", "Amazon.co.uk_ Best_All_Reviews***.csv")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)
print(df)

Resultant CSV after joining all CSV files at a particular location...
                                                      Text  \
0                        Andrex Gentle Clean Toilet Tissue   
1                        Andrex Gentle Clean Toilet Tissue   
2                        Andrex Gentle Clean Toilet Tissue   
3                        Andrex Gentle Clean Toilet Tissue   
4                        Andrex Gentle Clean Toilet Tissue   
...                                                    ...   
1019995  Garnier Hand Repair Intensive Restoring Hand C...   
1019996  Linda's Silicone Stove Gap Covers (2 Pack), He...   
1019997  Linda's Silicone Stove Gap Covers (2 Pack), He...   
1019998  Linda's Silicone Stove Gap Covers (2 Pack), He...   
1019999  Linda's Silicone Stove Gap Covers (2 Pack), He...   

                                                     Title            Name  \
0                             Fun Size C Wafer Thin Andrex        DM Bunny   
1             They've shrunk.

In [14]:
df.head()

,Text,Title,Name,aiconalt,View,asizemini,State,View1,asizebase
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0 out of 5 stars,Reviewed in the United Kingdom on 18 September...,Size Name: 45 Count (Pack of 1),Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0 out of 5 stars,Reviewed in the United Kingdom on 11 July 2019,Size Name: 45 Count (Pack of 1),Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0 out of 5 stars,Reviewed in the United Kingdom on 8 May 2019,Size Name: 45 Count (Pack of 1),Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0 out of 5 stars,Reviewed in the United Kingdom on 2 March 2018,Size Name: 45 Count (Pack of 1),Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0 out of 5 stars,Reviewed in the United Kingdom on 29 March 2018,Size Name: 45 Count (Pack of 1),Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful


In [15]:
df.rename(columns = {"Text":"Product", "Title":"Title of Review", "View":"Location and Date", "aiconalt":"Individual Review", "View1": "Review", "asizebase": "Helpful"}, inplace= True)

In [16]:
df.head()

,Product,Title of Review,Name,Individual Review,Location and Date,asizemini,State,Review,Helpful
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0 out of 5 stars,Reviewed in the United Kingdom on 18 September...,Size Name: 45 Count (Pack of 1),Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0 out of 5 stars,Reviewed in the United Kingdom on 11 July 2019,Size Name: 45 Count (Pack of 1),Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0 out of 5 stars,Reviewed in the United Kingdom on 8 May 2019,Size Name: 45 Count (Pack of 1),Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0 out of 5 stars,Reviewed in the United Kingdom on 2 March 2018,Size Name: 45 Count (Pack of 1),Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0 out of 5 stars,Reviewed in the United Kingdom on 29 March 2018,Size Name: 45 Count (Pack of 1),Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful


In [17]:
df.isna().sum()

Product                   0
Title of Review      305188
Name                    567
Individual Review       530
Location and Date       530
asizemini            238599
State                 28373
Review                 1273
Helpful              897123
dtype: int64

In [18]:
df.drop(columns="asizemini", inplace=True)

In [19]:
df.dropna(subset=["Review"], inplace=True)

In [20]:
df.isna().sum()

Product                   0
Title of Review      304341
Name                     37
Individual Review         0
Location and Date         0
State                 27824
Review                    0
Helpful              895934
dtype: int64

In [21]:
df.describe()

,Product,Title of Review,Name,Individual Review,Location and Date,State,Review,Helpful
count,1018727,714386,1018690,1018727,1018727,990903,1018727,122793
unique,950,333843,519420,5,27200,1,904448,415
top,Moleskine Classic Ruled Paper Notebook,Five Stars,Amazon Customer,5.0 out of 5 stars,Reviewed in the United Kingdom on 13 December ...,Verified Purchase,\n Good\n,One person found this helpful
freq,8327,47287,69286,724449,1025,990903,4430,68307


In [22]:
df[["Individual Review"]] = df[["Individual Review"]].apply(lambda x: x.str.rstrip(' out of 5 stars'))

In [23]:
df =  df.astype(str)


In [24]:
df.head()

,Product,Title of Review,Name,Individual Review,Location and Date,State,Review,Helpful
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,Reviewed in the United Kingdom on 18 September...,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,Reviewed in the United Kingdom on 11 July 2019,Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,Reviewed in the United Kingdom on 8 May 2019,Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,Reviewed in the United Kingdom on 2 March 2018,Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,Reviewed in the United Kingdom on 29 March 2018,Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful


In [25]:
df['Location and Date'] = df['Location and Date'].apply(lambda x: x.lstrip('Reviewed in the '))

In [26]:
df.head()

,Product,Title of Review,Name,Individual Review,Location and Date,State,Review,Helpful
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,United Kingdom on 18 September 2017,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,United Kingdom on 11 July 2019,Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,United Kingdom on 8 May 2019,Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,United Kingdom on 2 March 2018,Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,United Kingdom on 29 March 2018,Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful


In [27]:
df["Helpful"] = df['Helpful'].apply(lambda x: x.replace(',',''))

In [28]:
df.head()

,Product,Title of Review,Name,Individual Review,Location and Date,State,Review,Helpful
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,United Kingdom on 18 September 2017,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,United Kingdom on 11 July 2019,Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,United Kingdom on 8 May 2019,Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,United Kingdom on 2 March 2018,Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,United Kingdom on 29 March 2018,Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful


In [29]:
df["Date"] = df["Location and Date"]

In [30]:
df["Date"] = df["Date"].apply(lambda x: x.split(' on ')[-1].strip())

In [31]:
df.head()

,Product,Title of Review,Name,Individual Review,Location and Date,State,Review,Helpful,Date
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,United Kingdom on 18 September 2017,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful,18 September 2017
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,United Kingdom on 11 July 2019,Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful,11 July 2019
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,United Kingdom on 8 May 2019,Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful,8 May 2019
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,United Kingdom on 2 March 2018,Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful,2 March 2018
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,United Kingdom on 29 March 2018,Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful,29 March 2018


In [32]:
df["Location and Date"] = df["Location and Date"].apply(lambda x: x.split(' on ')[0].strip())

In [33]:
df.head()

,Product,Title of Review,Name,Individual Review,Location and Date,State,Review,Helpful,Date
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,United Kingdom,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful,18 September 2017
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,United Kingdom,Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful,11 July 2019
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,United Kingdom,Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful,8 May 2019
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,United Kingdom,Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful,2 March 2018
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,United Kingdom,Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful,29 March 2018


In [34]:
df.rename(columns={"Location and Date": "Location"}, inplace = True)

In [35]:
df.head()

,Product,Title of Review,Name,Individual Review,Location,State,Review,Helpful,Date
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,United Kingdom,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful,18 September 2017
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,United Kingdom,Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful,11 July 2019
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,United Kingdom,Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful,8 May 2019
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,United Kingdom,Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful,2 March 2018
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,United Kingdom,Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful,29 March 2018


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1018727 entries, 0 to 1019999
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   Product            1018727 non-null  object
 1   Title of Review    1018727 non-null  object
 2   Name               1018727 non-null  object
 3   Individual Review  1018727 non-null  object
 4   Location           1018727 non-null  object
 5   State              1018727 non-null  object
 6   Review             1018727 non-null  object
 7   Helpful            1018727 non-null  object
 8   Date               1018727 non-null  object
dtypes: object(9)
memory usage: 77.7+ MB


In [38]:
df[["day","month", "year"]] = df["Date"].apply(lambda x: pd.Series(x.split(' ')))

In [39]:
df["Date"] = pd.to_datetime(df['year'].astype(str)  + df['month'] + df["day"], format='%Y%B%d')

In [40]:
df.head()

,Product,Title of Review,Name,Individual Review,Location,State,Review,Helpful,Date,day,month,year
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,United Kingdom,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful,2017-09-18,18,September,2017
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,United Kingdom,Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful,2019-07-11,11,July,2019
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,United Kingdom,Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful,2019-05-08,8,May,2019
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,United Kingdom,Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful,2018-03-02,2,March,2018
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,United Kingdom,Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful,2018-03-29,29,March,2018


In [41]:
df.drop(columns = ["day", "month", "year"], inplace= True)

In [42]:
df["Helpful"].value_counts()

nan                              895934
One person found this helpful     68307
2 people found this helpful       18811
3 people found this helpful        8618
4 people found this helpful        4953
                                  ...  
406 people found this helpful         1
347 people found this helpful         1
281 people found this helpful         1
815 people found this helpful         1
289 people found this helpful         1
Name: Helpful, Length: 416, dtype: int64

In [43]:
df["Helpful"] = df["Helpful"].apply(lambda x: x.rstrip('people found this helpful'))

In [44]:
df["Helpful"].value_counts()

na         895934
One per     68307
2           18811
3            8618
4            4953
            ...  
362             1
281             1
1123            1
407             1
257             1
Name: Helpful, Length: 416, dtype: int64

In [45]:
df["Helpful"].replace(['One per', "na"], [1, 0], inplace = True)

In [46]:
df.head()

,Product,Title of Review,Name,Individual Review,Location,State,Review,Helpful,Date
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,United Kingdom,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593,2017-09-18
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,United Kingdom,Verified Purchase,"\n I've been ordering these for about a year,...",242,2019-07-11
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,United Kingdom,Verified Purchase,\n Ibought these after buying similar ones pr...,209,2019-05-08
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,United Kingdom,Verified Purchase,\n I have to admit that I just don't like And...,34,2018-03-02
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,United Kingdom,Verified Purchase,\n Not really a bargain as it comes a lot les...,216,2018-03-29


In [47]:
df[["Product", "Title of Review", "Name", "Location", "State", "Review"]].astype(str).dtypes
df[["Individual Review"]].astype(float)
df[["Helpful"]].astype(int)

,Helpful
0,593
1,242
2,209
3,34
4,216
...,...
1019995,0
1019996,0
1019997,0
1019998,0


In [48]:
df["Location"].value_counts()

United Kingdom    714459
United States      76238
Italy              65322
Germany            59078
Spain              43261
France             31253
Canada             16841
Mexico              5955
India               2268
Japan               1772
Australia           1025
Brazil               798
Netherlands          227
Sweden               153
Singapore             52
Poland                25
Name: Location, dtype: int64

In [49]:
df.describe()

<ipython-input-49-ea8415b8a3ee>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,Product,Title of Review,Name,Individual Review,Location,State,Review,Helpful,Date
count,1018727,1018727,1018727,1018727,1018727,1018727,1018727,1018727.0,1018727
unique,950,333844,519421,5,16,2,904448,416.0,4677
top,Moleskine Classic Ruled Paper Notebook,nan,Amazon Customer,5.0,United Kingdom,Verified Purchase,\n Good\n,0.0,2020-12-13 00:00:00
freq,8327,304341,69286,724449,714459,990903,4430,895934.0,1269
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003-12-15 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-13 00:00:00


In [50]:
(df == "NaN").all(axis=0)

Product              False
Title of Review      False
Name                 False
Individual Review    False
Location             False
State                False
Review               False
Helpful              False
Date                 False
dtype: bool

In [51]:
#greater than the start date and smaller than the end date
df_covd_dates = df[(df["Date"] > "2019-12-01") & (df['Date'] <= "2020-08-01")]

In [52]:
df_covd_dates.describe()

<ipython-input-52-41ec728bacd6>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_covd_dates.describe()


,Product,Title of Review,Name,Individual Review,Location,State,Review,Helpful,Date
count,155836,155836,155836,155836,155836,155836,155836,155836.0,155836
unique,738,60053,97059,5,14,2,141479,177.0,244
top,"Gritin Resistance Bands, [Set of 5] Skin-Frien...",nan,Amazon Customer,5.0,United Kingdom,Verified Purchase,\n Good\n,0.0,2020-01-02 00:00:00
freq,1579,48023,10532,113404,107830,153357,751,139911.0,1002
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-02 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-01 00:00:00


In [53]:
(df_covd_dates["Review"] == "NaN").value_counts()

False    155836
Name: Review, dtype: int64

In [54]:
df_covd_dates["Review"].value_counts()

\n  Good\n                                                                                                                                                                                                                                                                                                                                           751
\n  Great\n                                                                                                                                                                                                                                                                                                                                          478
\n  Great product\n                                                                                                                                                                                                                                                                                                   

In [55]:
df_covd_dates["Location"].value_counts()

United Kingdom    107830
United States      11639
Italy              11279
Germany             8516
Spain               7685
France              4578
Canada              2558
Mexico               997
India                308
Japan                240
Australia            138
Brazil                52
Singapore             10
Netherlands            6
Name: Location, dtype: int64

### Drop non English speaking countries

In [56]:
df_covd_dates = df_covd_dates.loc[df_covd_dates["Location"].isin(["United Kingdom", "United States", "Canada", "India", "Australia"])]

In [57]:
df_covd_dates["Location"].value_counts()

United Kingdom    107830
United States      11639
Canada              2558
India                308
Australia            138
Name: Location, dtype: int64

In [58]:
df_covd_dates.dtypes


Product                      object
Title of Review              object
Name                         object
Individual Review            object
Location                     object
State                        object
Review                       object
Helpful                      object
Date                 datetime64[ns]
dtype: object

In [59]:
df_covd_dates

,Product,Title of Review,Name,Individual Review,Location,State,Review,Helpful,Date
10,Andrex Gentle Clean Toilet Tissue,Overpriced nonsense,Mr R H Angus,2.0,United Kingdom,Verified Purchase,\n Why the change of price for those of us wh...,2,2020-03-19
15,Andrex Gentle Clean Toilet Tissue,Absolutely not worth the money,yc chai,1.0,United Kingdom,Verified Purchase,I never thought that I still need to read any ...,3,2020-05-30
21,Andrex Gentle Clean Toilet Tissue,CON,Customer in York,1.0,United Kingdom,Verified Purchase,\n We brought Andrex multi on a few occasions...,2,2020-01-05
24,Andrex Gentle Clean Toilet Tissue,Sad sad sad,El,1.0,United Kingdom,Verified Purchase,"\n Disappointed in these thinned out, less de...",2,2020-03-09
28,Andrex Gentle Clean Toilet Tissue,Beware - inferior quality,Kenkonti,1.0,United Kingdom,Verified Purchase,\n This product sums up the pitfalls of buyin...,2,2020-06-24
...,...,...,...,...,...,...,...,...,...
1019983,AnySharp Knife Sharpener with PowerGrip,Make your own razors,Cupid Stunt,5.0,United Kingdom,Verified Purchase,"\n This is proper, will take metal shavings o...",0,2020-01-24
1019992,Garnier Hand Repair Intensive Restoring Hand C...,Ideal product,Amazon Customer,5.0,United Kingdom,Verified Purchase,\n A****\n,0,2020-05-09
1019993,Garnier Hand Repair Intensive Restoring Hand C...,Good and Cheap,Paul Fadur,5.0,United Kingdom,Verified Purchase,\n Good and Cheap for value\n,0,2020-02-22
1019994,Garnier Hand Repair Intensive Restoring Hand C...,Repair dry hand very well,Paul Bell,5.0,United Kingdom,Verified Purchase,\n Very good hand cream\n,0,2020-02-18


In [62]:
df_covd_dates = df_covd_dates.drop(columns = ["Title of Review", "Name", "State"])
df_covd_dates

,Product,Individual Review,Location,Review,Helpful,Date
10,Andrex Gentle Clean Toilet Tissue,2.0,United Kingdom,\n Why the change of price for those of us wh...,2,2020-03-19
15,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,I never thought that I still need to read any ...,3,2020-05-30
21,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,\n We brought Andrex multi on a few occasions...,2,2020-01-05
24,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,"\n Disappointed in these thinned out, less de...",2,2020-03-09
28,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,\n This product sums up the pitfalls of buyin...,2,2020-06-24
...,...,...,...,...,...,...
1019983,AnySharp Knife Sharpener with PowerGrip,5.0,United Kingdom,"\n This is proper, will take metal shavings o...",0,2020-01-24
1019992,Garnier Hand Repair Intensive Restoring Hand C...,5.0,United Kingdom,\n A****\n,0,2020-05-09
1019993,Garnier Hand Repair Intensive Restoring Hand C...,5.0,United Kingdom,\n Good and Cheap for value\n,0,2020-02-22
1019994,Garnier Hand Repair Intensive Restoring Hand C...,5.0,United Kingdom,\n Very good hand cream\n,0,2020-02-18


In [63]:
df_covd_dates.reset_index(drop=True, inplace=True)
df_covd_dates

,Product,Individual Review,Location,Review,Helpful,Date
0,Andrex Gentle Clean Toilet Tissue,2.0,United Kingdom,\n Why the change of price for those of us wh...,2,2020-03-19
1,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,I never thought that I still need to read any ...,3,2020-05-30
2,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,\n We brought Andrex multi on a few occasions...,2,2020-01-05
3,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,"\n Disappointed in these thinned out, less de...",2,2020-03-09
4,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,\n This product sums up the pitfalls of buyin...,2,2020-06-24
...,...,...,...,...,...,...
122468,AnySharp Knife Sharpener with PowerGrip,5.0,United Kingdom,"\n This is proper, will take metal shavings o...",0,2020-01-24
122469,Garnier Hand Repair Intensive Restoring Hand C...,5.0,United Kingdom,\n A****\n,0,2020-05-09
122470,Garnier Hand Repair Intensive Restoring Hand C...,5.0,United Kingdom,\n Good and Cheap for value\n,0,2020-02-22
122471,Garnier Hand Repair Intensive Restoring Hand C...,5.0,United Kingdom,\n Very good hand cream\n,0,2020-02-18


In [69]:
for idx, chunk in enumerate(np.array_split(df_covd_dates, 6)):
    chunk.to_csv(f'best_products_for_sentimental_{idx}.csv')

In [61]:
#df_covd_dates.to_csv('best_products_for_sentimental.csv')